In [90]:
#import necessary libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import xarray as xr
import scipy

import regionmask

# import regionmask

from scipy import stats
from scipy.stats import pearsonr, linregress

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd

import time
from datetime import datetime, date

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.ticker import MaxNLocator, FuncFormatter
import matplotlib.dates as mdates

import shapefile
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
import os

import sys
import os.path
import subprocess
import math
import csv

from netCDF4 import Dataset

#import s3fs
import copy
import json, copy
from pandas import DataFrame
from osgeo import gdal, ogr, osr

In [2]:
# Check if the file was properly saved at the given location (navigate to the folder) and open it from there again
gsfc_halfdegree_gms = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\GSFC\Data\gsfc.glb_.200204_202406_rl06v2.0_obp-ice6gd_halfdegree.nc')
jpl_halfdegree_gms = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\GSFC\JPL\GRCTellus.JPL.200204_202408.GLO.RL06.3M.MSCNv04CRI.nc')
csr_quarterdegree_gms = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\GSFC\CSR\CSR_GRACE_GRACE-FO_RL0603_Mascons_all-corrections.nc')

In [3]:
gsfc_halfdegree_gms

<xarray.Dataset> Size: 487MB
Dimensions:        (lon: 720, lat: 360, time: 234, bounds: 2)
Coordinates:
  * lon            (lon) float64 6kB 0.25 0.75 1.25 1.75 ... 358.8 359.2 359.8
  * lat            (lat) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * time           (time) datetime64[ns] 2kB 2002-04-18 ... 2024-06-16
Dimensions without coordinates: bounds
Data variables:
    land_mask      (lat, lon) float64 2MB ...
    lwe_thickness  (time, lat, lon) float64 485MB ...
    lon_bounds     (lon, bounds) float64 12kB ...
    lat_bounds     (lat, bounds) float64 6kB ...
    time_bounds    (time, bounds) datetime64[ns] 4kB ...
Attributes: (12/25)
    title:                      NASA GSFC GRACE and GRACE-FO MASCON RL06 v1.0
    summary:                    Monthly gravity solutions from GRACE and GRAC...
    platform:                   GRACE and GRACE-FO
    creator_name:               Bryant Loomis
    creator_email:              bryant.d.loomis@nasa.gov
    creator_url:                https://earth.gsfc.nasa.gov/geo/data/grace-ma...
    ...                         ...
    GIA_removed:                ICE6G-D; Peltier, W. R., D. F. Argus, and R. ...
    geocenter_correction:       JPL TN-13
    C_20_substitution:          TN-14; Loomis et al., 2020, Geophys. Res. Let...
    C_30_substitution:          TN-14; Loomis et al., 2020, Geophys. Res. Let...
    journal_reference:          Loomis et al. 2019, J. Geod., https://doi.org...
    date_created:               date_stamp

D:\MSRSGI\Masters_thesis\Data\GLDAS_Data\From_2002__2016

In [91]:
# Adjust the path and pattern as needed
ds = xr.open_mfdataset(
    r'D:/MSRSGI/Masters_thesis/Data/GLDAS_Data/From_2002__2016/*.nc',
    combine='nested',
    concat_dim='time',
    data_vars='minimal',
    coords='minimal',
    compat='override',
    parallel=True
)
ds

<xarray.Dataset> Size: 68MB
Dimensions:      (time: 158, lat: 150, lon: 360, bounds: 2)
Coordinates:
  * lon          (lon) float64 3kB -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * lat          (lat) float64 1kB -59.5 -58.5 -57.5 -56.5 ... 87.5 88.5 89.5
  * time         (time) datetime64[ns] 1kB 2002-04-17T12:00:00 ... 2016-12-24...
Dimensions without coordinates: bounds
Data variables:
    TWS_monthly  (time, lat, lon) float64 68MB dask.array<chunksize=(1, 150, 360), meta=np.ndarray>
    lat_bounds   (lat, bounds) float64 2kB dask.array<chunksize=(150, 2), meta=np.ndarray>
    lon_bounds   (lon, bounds) float64 6kB dask.array<chunksize=(360, 2), meta=np.ndarray>
    time_bounds  (time, bounds) datetime64[ns] 3kB dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/46)
    conventions:                CF-1.6,ACDD-1.3,ISO 8601
    metadata_Conventions:       Unidata Dataset Discovery v1.0
    standard_name_vocabulary:   CF-1.6
    geospatial_lat_min:         -59.5
    geospatial_lat_max:         89.5
    geospatial_lat_units:       degrees_north
    ...                         ...
    date_created:               2020-03-20T05:49:31
    time_epoch:                 2002-01-01T00:00:00 UTC
    naming_authority:           org.doi.dx
    time_coverage_start:        2002-04-04T00:00:00
    time_coverage_end:          2002-05-01T00:00:00
    unused_days:                10-Apr-2002 , 11-Apr-2002 , 28-Apr-2002

In [92]:
ds.TWS_monthly.values

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

In [93]:
ds.to_netcdf('GLDAS_2002_2016.nc')

In [94]:
gldas_combined = xr.open_dataset('GLDAS_2002_2016.nc')
gldas_combined

<xarray.Dataset> Size: 68MB
Dimensions:      (time: 158, lat: 150, lon: 360, bounds: 2)
Coordinates:
  * lon          (lon) float64 3kB -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * lat          (lat) float64 1kB -59.5 -58.5 -57.5 -56.5 ... 87.5 88.5 89.5
  * time         (time) datetime64[ns] 1kB 2002-04-17T12:00:00 ... 2016-12-24...
Dimensions without coordinates: bounds
Data variables:
    TWS_monthly  (time, lat, lon) float64 68MB ...
    lat_bounds   (lat, bounds) float64 2kB ...
    lon_bounds   (lon, bounds) float64 6kB ...
    time_bounds  (time, bounds) datetime64[ns] 3kB ...
Attributes: (12/46)
    conventions:                CF-1.6,ACDD-1.3,ISO 8601
    metadata_Conventions:       Unidata Dataset Discovery v1.0
    standard_name_vocabulary:   CF-1.6
    geospatial_lat_min:         -59.5
    geospatial_lat_max:         89.5
    geospatial_lat_units:       degrees_north
    ...                         ...
    date_created:               2020-03-20T05:49:31
    time_epoch:                 2002-01-01T00:00:00 UTC
    naming_authority:           org.doi.dx
    time_coverage_start:        2002-04-04T00:00:00
    time_coverage_end:          2002-05-01T00:00:00
    unused_days:                10-Apr-2002 , 11-Apr-2002 , 28-Apr-2002

In [95]:
df_gldas = gldas_combined.TWS_monthly.to_dataframe()
df_gldas

TWS_monthly
time                lat   lon                
2002-04-17 12:00:00 -59.5 -179.5          NaN
                          -178.5          NaN
                          -177.5          NaN
                          -176.5          NaN
                          -175.5          NaN
...                                       ...
2016-12-24 12:00:00  89.5  175.5          NaN
                           176.5          NaN
                           177.5          NaN
                           178.5          NaN
                           179.5          NaN

[8532000 rows x 1 columns]

In [96]:
df_gldas.fillna(0, inplace=True)

In [97]:
df_gldas

TWS_monthly
time                lat   lon                
2002-04-17 12:00:00 -59.5 -179.5          0.0
                          -178.5          0.0
                          -177.5          0.0
                          -176.5          0.0
                          -175.5          0.0
...                                       ...
2016-12-24 12:00:00  89.5  175.5          0.0
                           176.5          0.0
                           177.5          0.0
                           178.5          0.0
                           179.5          0.0

[8532000 rows x 1 columns]

In [98]:
# Assuming df_costg is your DataFrame
df_gldas_back2nc = xr.Dataset.from_dataframe(df_gldas)
df_gldas_back2nc.to_netcdf("gldas.nc")
gldas_back2nc_tws= xr.open_dataset('gldas.nc')
gldas_back2nc_tws

<xarray.Dataset> Size: 68MB
Dimensions:      (time: 158, lat: 150, lon: 360)
Coordinates:
  * time         (time) datetime64[ns] 1kB 2002-04-17T12:00:00 ... 2016-12-24...
  * lat          (lat) float64 1kB -59.5 -58.5 -57.5 -56.5 ... 87.5 88.5 89.5
  * lon          (lon) float64 3kB -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
Data variables:
    TWS_monthly  (time, lat, lon) float64 68MB ...

In [99]:
gldas_back2nc_tws.TWS_monthly.values

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [100]:
# Target latitude: 89.5 to -90.0 (inclusive), step -0.5
target_lat = np.arange(89.5, -90.5, -0.5)

# Target longitude: 0.0 to 359.5 (inclusive), step 0.5
target_lon = np.arange(0.0, 360.0, 0.5)

ds_halfdeg = gldas_back2nc_tws.interp(lat=target_lat, lon=target_lon) #same interpolation as costg from 1 degree grid to half degree grid
ds_halfdeg.to_netcdf("linearinterpolation_GLDAS_tws_half_degree.nc")

In [101]:
gldas_halfdeg = xr.open_dataset('linearinterpolation_GLDAS_tws_half_degree.nc')
gldas_halfdeg

<xarray.Dataset> Size: 328MB
Dimensions:      (time: 158, lat: 360, lon: 720)
Coordinates:
  * time         (time) datetime64[ns] 1kB 2002-04-17T12:00:00 ... 2016-12-24...
  * lat          (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -89.0 -89.5 -90.0
  * lon          (lon) float64 6kB 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
Data variables:
    TWS_monthly  (time, lat, lon) float64 328MB ...

In [102]:
ds_gldas = gldas_halfdeg.TWS_monthly
ds_gldas

<xarray.DataArray 'TWS_monthly' (time: 158, lat: 360, lon: 720)> Size: 328MB
[40953600 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-17T12:00:00 ... 2016-12-24T12:...
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5

In [103]:
ds_gldas_rename = ds_gldas.rename('lwe_thickness')
ds_gldas_rename

<xarray.DataArray 'lwe_thickness' (time: 158, lat: 360, lon: 720)> Size: 328MB
[40953600 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-17T12:00:00 ... 2016-12-24T12:...
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5

In [104]:
# Select the time range from April 2002 to April 2022
subset_gldas_halfdegree = ds_gldas_rename.sel(time=slice("2002-04", "2016-03"))
subset_gldas_halfdegree

<xarray.DataArray 'lwe_thickness' (time: 152, lat: 360, lon: 720)> Size: 315MB
[39398400 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-17T12:00:00 ... 2016-03-16T12:...
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5

In [105]:
gldas_ds = subset_gldas_halfdegree
gldas_ds = gldas_ds.resample(time='M').mean().interpolate_na(dim='time')
gldas_ds #mm native/original unit

C:\Users\sudur\miniconda3\envs\Thesis\Lib\site-packages\xarray\groupers.py:498: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.DataArray 'lwe_thickness' (time: 168, lat: 360, lon: 720)> Size: 348MB
array([[[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], shape=(168, 360, 720))
Coordinates:
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31

In [106]:
nan_gldas = gldas_ds.to_dataframe()
nan_gldas

lwe_thickness
time       lat   lon                 
2002-04-30  89.5 0.0              0.0
                 0.5              0.0
                 1.0              0.0
                 1.5              0.0
                 2.0              0.0
...                               ...
2016-03-31 -90.0 357.5            NaN
                 358.0            NaN
                 358.5            NaN
                 359.0            NaN
                 359.5            NaN

[43545600 rows x 1 columns]

In [109]:
nan_gldas_filled = nan_gldas.fillna(0)

In [110]:
nan_gldas_filled

lwe_thickness
time       lat   lon                 
2002-04-30  89.5 0.0              0.0
                 0.5              0.0
                 1.0              0.0
                 1.5              0.0
                 2.0              0.0
...                               ...
2016-03-31 -90.0 357.5            0.0
                 358.0            0.0
                 358.5            0.0
                 359.0            0.0
                 359.5            0.0

[43545600 rows x 1 columns]

In [111]:
gldas_back2nc = xr.Dataset.from_dataframe(nan_gldas_filled)
gldas_back2nc.to_netcdf("gldas_linearinterpolationaandfill0.nc")
gldas_back2nc_tws= xr.open_dataset('gldas_linearinterpolationaandfill0.nc')
gldas_back2nc_tws

<xarray.Dataset> Size: 348MB
Dimensions:        (time: 168, lat: 360, lon: 720)
Coordinates:
  * time           (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
  * lat            (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -89.0 -89.5 -90.0
  * lon            (lon) float64 6kB 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
Data variables:
    lwe_thickness  (time, lat, lon) float64 348MB ...

In [114]:
gldas_ds = gldas_back2nc_tws.lwe_thickness
gldas_ds 

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 360, lon: 720)> Size: 348MB
[43545600 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5

In [115]:
gldas_ds_meter_ = gldas_ds / 1000
gldas_ds_meter_.attrs['units'] = 'meters'  # Optional but good practice
gldas_ds_meter_

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 360, lon: 720)> Size: 348MB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], shape=(168, 360, 720))
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
Attributes:
    units:    meters

In [116]:
gldas_ds_meter_.to_netcdf(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\New_TWS_dueto_HydrologicalSignal_meter.nc')

In [117]:
gldas_tws_meter = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\New_TWS_dueto_HydrologicalSignal_meter.nc')
gldas_tws_meter

<xarray.Dataset> Size: 348MB
Dimensions:        (time: 168, lat: 360, lon: 720)
Coordinates:
  * time           (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
  * lat            (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -89.0 -89.5 -90.0
  * lon            (lon) float64 6kB 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
Data variables:
    lwe_thickness  (time, lat, lon) float64 348MB ...

In [29]:
# gldas_tws_meter.lwe_thickness.values

------
-------

In [118]:
# 1. Load and prepare the shapefile
gdf = gpd.read_file(r'D:\MSRSGI\Masters_thesis\Data\HMA_Region_Shapefiles\Regions_shapefiles\New folder\Updated_TotalHMA_boundary1_22.shp')
gdf_4326 = gdf.to_crs('EPSG:4326')  # Reproject to match dataset CRS

dropped_gdf_4326 = gdf_4326.drop(['Primary__1'], axis=1)
dropped_gdf_4326

,Primary_ID,Area,Shape_Area,Region_No,geometry
0,Eastern Hindu Kush,111147.061871,1.111471e+11,8,"POLYGON ((73.45158 36.98397, 73.44585 36.96572..."
1,Western Himalaya,211514.200081,2.115142e+11,10,"POLYGON ((75.78739 35.3156, 75.7874 35.3156, 7..."
2,Eastern Himalaya,166943.561268,1.669436e+11,12,"POLYGON ((95.22303 29.82187, 95.21352 29.80717..."
3,Central Himalaya,270268.094462,2.702681e+11,11,"POLYGON ((80.93559 30.70763, 80.98095 30.6962,..."
4,Karakoram,92830.485352,9.283049e+10,9,"POLYGON ((75.87358 37.3118, 75.89056 37.31113,..."
5,Western Pamir,124880.500694,1.248805e+11,6,"POLYGON ((73.91513 39.69697, 73.92109 39.68454..."
6,Pamir Alay,83444.425531,8.344443e+10,5,"POLYGON ((73.66215 40.58146, 73.727 40.56312, ..."
7,Northern/Western Tien Shan,205724.916795,2.057249e+11,4,"POLYGON ((83.06636 43.4915, 82.95135 43.44983,..."
8,Dzhungarsky Alatau,39543.166059,3.954317e+10,1,"POLYGON ((81.62713 44.61643, 81.28999 44.60156..."
9,Western Kunlun Shan,132813.049169,1.328130e+11,13,"POLYGON ((77.76951 37.37377, 78.24539 37.24774..."


In [119]:
gdf1 = gpd.read_file(r"D:\MSRSGI\Masters_thesis\Data\HMA_Region_Shapefiles\Regions_shapefiles\New folder\Updated_SingleHMAboundarythatincludes1to22.shp")
gdf1_4326 = gdf1.to_crs('EPSG:4326')  # Reproject to match dataset CRS

dropped_gdf1_4326 = gdf1_4326.drop(['Primary__1'], axis=1)  #already done , so comes weror wehn done
dropped_gdf1_4326

,Primary_ID,Area,Shape_Area,Region_No,geometry
0,High Mountain Asia Region,3.993978e+06,3.993978e+12,23,"POLYGON ((92.26394 39.3763, 92.24847 39.32136,..."


In [120]:
gdf_concat = pd.concat([dropped_gdf_4326, dropped_gdf1_4326 ], axis=0, ignore_index=True)
gdf_concat

,Primary_ID,Area,Shape_Area,Region_No,geometry
0,Eastern Hindu Kush,1.111471e+05,1.111471e+11,8,"POLYGON ((73.45158 36.98397, 73.44585 36.96572..."
1,Western Himalaya,2.115142e+05,2.115142e+11,10,"POLYGON ((75.78739 35.3156, 75.7874 35.3156, 7..."
2,Eastern Himalaya,1.669436e+05,1.669436e+11,12,"POLYGON ((95.22303 29.82187, 95.21352 29.80717..."
3,Central Himalaya,2.702681e+05,2.702681e+11,11,"POLYGON ((80.93559 30.70763, 80.98095 30.6962,..."
4,Karakoram,9.283049e+04,9.283049e+10,9,"POLYGON ((75.87358 37.3118, 75.89056 37.31113,..."
5,Western Pamir,1.248805e+05,1.248805e+11,6,"POLYGON ((73.91513 39.69697, 73.92109 39.68454..."
6,Pamir Alay,8.344443e+04,8.344443e+10,5,"POLYGON ((73.66215 40.58146, 73.727 40.56312, ..."
7,Northern/Western Tien Shan,2.057249e+05,2.057249e+11,4,"POLYGON ((83.06636 43.4915, 82.95135 43.44983,..."
8,Dzhungarsky Alatau,3.954317e+04,3.954317e+10,1,"POLYGON ((81.62713 44.61643, 81.28999 44.60156..."
9,Western Kunlun Shan,1.328130e+05,1.328130e+11,13,"POLYGON ((77.76951 37.37377, 78.24539 37.24774..."


In [121]:
ds = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\New_TWS_dueto_HydrologicalSignal_meter.nc')

da = ds['lwe_thickness']  # Get the DataArray
da

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 360, lon: 720)> Size: 348MB
[43545600 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
  * lat      (lat) float64 3kB 89.5 89.0 88.5 88.0 ... -88.5 -89.0 -89.5 -90.0
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
Attributes:
    units:    meters

In [123]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray
from shapely.geometry import mapping

# # 1. Load and prepare the shapefile
# gdf = gpd.read_file(r'D:/MSRSGI/Masters_thesis/Data/HMA_Region_Shapefiles/Regions_shapefiles/HMA_region_annoted_final.shp')
# gdf_4326 = gdf.to_crs('EPSG:4326')  # Reproject to match dataset CRS

# 2. Load the lwe_thickness dataset 
ds = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\New_TWS_dueto_HydrologicalSignal_meter.nc')
da = ds['lwe_thickness']  # Get the DataArray

# xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\TWS_dueto_HydrologicalSignal_meter.nc')

# 3. Define the provided functions
def area_weighting(grid_resolution):
    """
    Calculate the area of each grid, globally, corresponding to the latitudes and longitudes.
    """
    longitude_grid = np.linspace(0, 359+(1-grid_resolution), int(360/grid_resolution), dtype='float')
    latitude_grid = np.linspace(0, 179+(1-grid_resolution), int(180/grid_resolution), dtype='float')
    longitude_grid1 = np.linspace(1*grid_resolution, 360, int(360/grid_resolution), dtype='float')
    latitude_grid1 = np.linspace(1*grid_resolution, 180, int(180/grid_resolution), dtype='float')
    
    lambd,theta = np.meshgrid(longitude_grid, latitude_grid)  
    lambd1,theta1 = np.meshgrid(longitude_grid1, latitude_grid1)
    
    delta_latitude = np.sin(np.deg2rad(90-theta)) - np.sin(np.deg2rad(90-theta1))
    delta_longitude = (lambd1 - lambd) * np.pi/180

    # Area of each grid in m²
    area = (6378.137**2) * pow(10,6) * (np.multiply(delta_latitude, delta_longitude))
    return area

def Basinaverage(temp, gs, shp_basin, basin_area):
    """
    Calculate the basin average of the total water storage (TWS).
    """
    # Apply area weighting to the data
    temp_weighted = temp.copy()
    temp_weighted = temp * area_weighting(gs)  # Convert to volume
    
    # Set spatial dimensions and CRS
    basin_tws = temp_weighted.rio.write_crs("EPSG:4326", inplace=True)
    basin_tws = basin_tws.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
            
    # Clip data to basin shape
    basin_tws = basin_tws.rio.clip(shp_basin.geometry.apply(mapping), shp_basin.crs, drop=True)
    
    # Calculate basin average TWS
    basin_avg_tws = basin_tws.sum(dim=('lon','lat'), skipna=True) / basin_area
    
    # Clean up spatial references
    if 'spatial_ref' in basin_avg_tws.coords:
        basin_avg_tws = basin_avg_tws.drop_vars('spatial_ref')
    
    return basin_avg_tws

# 4. Process each region
results = []
grid_resolution = 0.5  # Grid resolution in degrees

#for idx, region in gdf_4326.iterrows():
for idx, region in gdf_concat.iterrows():
    region_name = region.Primary_ID
    region_area = region.Shape_Area  # Exact region area from shapefile
    number_region = region.Region_No #latest
    
    #  GeoDataFrame
    region_gdf = gpd.GeoDataFrame({
        'geometry': [region.geometry],
        'Primary_ID': [region_name],
        'Region_No': [number_region],
        'Shape_Area': [region_area]
    }, crs=gdf_concat.crs)   #gdf_4326 is replaced by gdf_concat
    
    # Calculate the basin average using the reference function
    basin_avg = Basinaverage(da, grid_resolution, region_gdf, region_area)
    
    # # Add region metadata
    # basin_avg = basin_avg.assign_attrs({
    #     'region_name': region_name,
    #     #'region_area_m2': float(region_area),
    #     'region_area_m2': region_area,
    #     'region_id': number_region
    # })

    meta = xr.Dataset({
    'region_id': result_ds.region_id,
    'region_area_m2': result_ds.region_area_m2}, coords={'region': result_ds.region})

    # print(meta.to_dataframe())
    
    basin_avg = basin_avg.expand_dims(region=[region_name])
    basin_avg = basin_avg.assign_coords({
    'region_area_m2': ('region', [region_area]),
    'region_id': ('region', [number_region])})
    results.append(basin_avg)

# 5. Combine all regions into a single dataset
result_ds = xr.concat(results, dim='region')

In [124]:
result_ds

<xarray.DataArray 'lwe_thickness' (region: 23, time: 168)> Size: 31kB
array([[ 0.00034983, -0.00248498, -0.02700681, ...,  0.03742564,
         0.05749489,  0.10258223],
       [-0.00488452, -0.01794971, -0.02671555, ...,  0.01527956,
         0.0344709 ,  0.07782283],
       [-0.06431238,  0.01029791,  0.04878636, ..., -0.05411531,
        -0.04489925, -0.03299313],
       ...,
       [ 0.02089722,  0.0240949 ,  0.02059231, ..., -0.00055861,
         0.00448447,  0.00837358],
       [-0.00177903, -0.01180823, -0.0223231 , ..., -0.01252386,
        -0.00803285, -0.00303564],
       [-0.01015769, -0.00068186,  0.00571574, ..., -0.01486679,
        -0.01038868, -0.00027629]], shape=(23, 168))
Coordinates:
  * region          (region) object 184B 'Eastern Hindu Kush' ... 'High Mount...
  * time            (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
    region_area_m2  (region) float64 184B 1.111e+11 2.115e+11 ... 3.994e+12
    region_id       (region) int64 184B 8 10 12 11 9 6 5 4 ... 22 14 15 2 3 7 23

In [125]:
result_ds_mm = result_ds * 1000 #GLDAS derived hydrological signal: from m to mm
result_ds_mm

<xarray.DataArray 'lwe_thickness' (region: 23, time: 168)> Size: 31kB
array([[  0.34982701,  -2.4849812 , -27.00681065, ...,  37.42563953,
         57.49488969, 102.58223181],
       [ -4.88452468, -17.94971388, -26.71555053, ...,  15.27956104,
         34.47089932,  77.82283407],
       [-64.31238131,  10.29790602,  48.78636339, ..., -54.11530845,
        -44.89924567, -32.99313387],
       ...,
       [ 20.89722276,  24.09489798,  20.59231196, ...,  -0.55861162,
          4.48446768,   8.37358385],
       [ -1.77902798, -11.8082285 , -22.32310132, ..., -12.52385894,
         -8.0328544 ,  -3.03563891],
       [-10.15769452,  -0.68185841,   5.71574467, ..., -14.86679472,
        -10.38867601,  -0.2762855 ]], shape=(23, 168))
Coordinates:
  * region          (region) object 184B 'Eastern Hindu Kush' ... 'High Mount...
  * time            (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
    region_area_m2  (region) float64 184B 1.111e+11 2.115e+11 ... 3.994e+12
    region_id       (region) int64 184B 8 10 12 11 9 6 5 4 ... 22 14 15 2 3 7 23

In [126]:
result_ds_mm.attrs['units'] = 'mm'
result_ds_mm

<xarray.DataArray 'lwe_thickness' (region: 23, time: 168)> Size: 31kB
array([[  0.34982701,  -2.4849812 , -27.00681065, ...,  37.42563953,
         57.49488969, 102.58223181],
       [ -4.88452468, -17.94971388, -26.71555053, ...,  15.27956104,
         34.47089932,  77.82283407],
       [-64.31238131,  10.29790602,  48.78636339, ..., -54.11530845,
        -44.89924567, -32.99313387],
       ...,
       [ 20.89722276,  24.09489798,  20.59231196, ...,  -0.55861162,
          4.48446768,   8.37358385],
       [ -1.77902798, -11.8082285 , -22.32310132, ..., -12.52385894,
         -8.0328544 ,  -3.03563891],
       [-10.15769452,  -0.68185841,   5.71574467, ..., -14.86679472,
        -10.38867601,  -0.2762855 ]], shape=(23, 168))
Coordinates:
  * region          (region) object 184B 'Eastern Hindu Kush' ... 'High Mount...
  * time            (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
    region_area_m2  (region) float64 184B 1.111e+11 2.115e+11 ... 3.994e+12
    region_id       (region) int64 184B 8 10 12 11 9 6 5 4 ... 22 14 15 2 3 7 23
Attributes:
    units:    mm

In [127]:
meta.to_dataframe()

,region_area_m2,region_id
region,,
Eastern Hindu Kush,1.111471e+11,8
Western Himalaya,2.115142e+11,10
Eastern Himalaya,1.669436e+11,12
Central Himalaya,2.702681e+11,11
Karakoram,9.283049e+10,9
Western Pamir,1.248805e+11,6
Pamir Alay,8.344443e+10,5
Northern/Western Tien Shan,2.057249e+11,4
Dzhungarsky Alatau,3.954317e+10,1


In [128]:
# 6. Save to NetCDF
result_ds_mm.to_netcdf(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New_HMA_regional_averaged_hydrologicalsignal_intermsofTWSA.nc')

In [129]:
# Load the dataset
ds = xr.open_dataset(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New_HMA_regional_averaged_hydrologicalsignal_intermsofTWSA.nc')

# Rearrange dimensions to (time, region)
da = ds['lwe_thickness'].transpose('time', 'region')

# Convert to pandas DataFrame
df = da.to_pandas()  # time as index, region names as columns
df

region,Eastern Hindu Kush,Western Himalaya,Eastern Himalaya,Central Himalaya,Karakoram,Western Pamir,Pamir Alay,Northern/Western Tien Shan,Dzhungarsky Alatau,Western Kunlun Shan,...,Tibetan Interior Mountains,Tanggula Shan,Eastern Tibetan Mountains,Qilian Shan,Eastern Kunlun Shan,Altun Shan,Eastern Tien Shan,Central Tien Shan,Eastern Pamir,High Mountain Asia Region
time,,,,,,,,,,,,,,,,,,,,,
2002-04-30,0.349827,-4.884525,-64.312381,-34.053981,-49.456800,60.770463,100.182423,81.403751,49.798154,-36.993517,...,-8.064644,-0.439720,-44.457635,-25.906150,-21.804769,-15.453217,-2.316336,20.897223,-1.779028,-10.157695
2002-05-31,-2.484981,-17.949714,10.297906,-29.092571,-53.989861,69.109297,105.403705,97.896336,63.738700,-37.810691,...,-6.371970,3.723981,-34.439601,-16.504097,-18.654586,-14.020893,2.706426,24.094898,-11.808229,-0.681858
2002-06-30,-27.006811,-26.715551,48.786363,12.540313,-67.876365,22.805606,57.136383,58.805402,39.191567,-35.933815,...,9.344392,19.186742,-33.094827,-18.265096,-15.869719,-14.118449,0.552680,20.592312,-22.323101,5.715745
2002-07-31,-52.346034,-35.773582,88.557769,55.560959,-82.225753,-25.041541,7.260150,18.411437,13.826196,-33.994376,...,25.584633,35.164928,-31.705228,-20.084794,-12.992023,-14.219256,-1.672858,16.972973,-33.188470,12.326601
2002-08-31,-77.685258,-44.831613,128.329175,98.581605,-96.575141,-72.888688,-42.616083,-21.982529,-11.539176,-32.054937,...,41.824874,51.143115,-30.315629,-21.904493,-10.114327,-14.320064,-3.898396,13.353634,-44.053838,18.937458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,6.993324,11.910621,-12.611316,-40.226968,47.293530,-21.873997,-3.972970,18.040203,25.006768,44.029993,...,-30.721423,-51.833822,-18.816299,7.767519,-5.941410,14.260944,22.124470,-4.505814,-6.885456,-8.062713
2015-12-31,24.853423,9.587918,-54.093948,-80.763792,64.683979,0.849908,22.998345,48.982943,55.077683,42.886585,...,-39.174400,-68.120314,-25.778699,6.930717,-9.274891,14.214132,30.611237,-1.092826,-9.798920,-16.054680
2016-01-31,37.425640,15.279561,-54.115308,-90.529023,75.891843,21.177035,46.606490,64.401631,70.330366,41.840328,...,-39.663943,-67.201451,-26.981542,5.913489,-9.792152,13.132943,32.853559,-0.558612,-12.523859,-14.866795


In [130]:
# Your region metadata
region_metadata_s = pd.DataFrame({
    'region': [
        'Eastern Hindu Kush', 'Western Himalaya', 'Eastern Himalaya', 'Central Himalaya', 'Karakoram',
        'Western Pamir', 'Pamir Alay', 'Northern/Western Tien Shan', 'Dzhungarsky Alatau', 'Western Kunlun Shan', 'Nyainqentanglha', 'Gangdise Mountains', 'Hengduan Shan',
        'Tibetan Interior Mountains', 'Tanggula Shan', 'Eastern Tibetan Mountains', 'Qilian Shan', 'Eastern Kunlun Shan', 'Altun Shan',
        'Eastern Tien Shan', 'Central Tien Shan', 'Eastern Pamir', 'High Mountain Asia Region'
    ],
    'region_id': [8, 10, 12, 11, 9, 6, 5, 4, 1, 13, 18, 17, 20, 16, 19, 21, 22, 14, 15, 2, 3, 7, 23]
})

# Sort by region_id
region_metadata_sorted_s = region_metadata_s.sort_values(by='region_id')
sorted_columns_s = region_metadata_sorted_s['region'].tolist()

df_s = df[sorted_columns_s]

df_s.index.name = 'time'
df_s.to_csv(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New_HMA_RegionalAvergaed_hydrologicalsignalTWS.csv', index=True)